# REMIND Electricity Sector Biosphere Flows

Calculate all biosphere flows for the REMIND electricity sector, both on a per-year and aggregated level.

In [16]:
%run initialize_notebook.ipynb

In [138]:
%run REMIND_Functions_to_modify_ecoinvent.ipynb

/home/al/miniconda3/envs/rmnd-lca/lib/python3.6/site-packages/constructive_geometries/cg.py:77: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


Geomatcher: Used 'AU' for 'AUS'
Geomatcher: Used 'CA' for 'CAN'
Geomatcher: Used 'HM' for 'HMD'
Geomatcher: Used 'NZ' for 'NZL'
Geomatcher: Used 'PM' for 'SPM'
Geomatcher: Used 'CN' for 'CHN'
Geomatcher: Used 'HK' for 'HKG'
Geomatcher: Used 'MO' for 'MAC'
Geomatcher: Used 'TW' for 'TWN'
Geomatcher: Used 'AX' for 'ALA'
Geomatcher: Used 'AT' for 'AUT'
Geomatcher: Used 'BE' for 'BEL'
Geomatcher: Used 'BG' for 'BGR'
Geomatcher: Used 'CY' for 'CYP'
Geomatcher: Used 'CZ' for 'CZE'
Geomatcher: Used 'DE' for 'DEU'
Geomatcher: Used 'DK' for 'DNK'
Geomatcher: Used 'ES' for 'ESP'
Geomatcher: Used 'EE' for 'EST'
Geomatcher: Used 'FI' for 'FIN'
Geomatcher: Used 'FR' for 'FRA'
Geomatcher: Used 'FO' for 'FRO'
Geomatcher: Used 'GB' for 'GBR'
Geomatcher: Used 'GI' for 'GIB'
Geomatcher: Used 'GR' for 'GRC'
Geomatcher: Used 'HR' for 'HRV'
Geomatcher: Used 'HU' for 'HUN'
Geomatcher: Used 'IM' for 'IMN'
Geomatcher: Used 'IE' for 'IRL'
Geomatcher: Used 'IT' for 'ITA'
Geomatcher: Used 'LT' for 'LTU'
Geomatch

In [18]:
import_karma()

Database Carma CCS already present.


In [109]:
databases

Databases dictionary with 3 object(s):
	Carma CCS
	biosphere3
	ecoinvent_3.5

In [78]:
names = available_electricity_generating_technologies["Wind"]

In [79]:
names

['electricity production, wind, <1MW turbine, onshore',
 'electricity production, wind, 1-3MW turbine, onshore',
 'electricity production, wind, >3MW turbine, onshore',
 'electricity production, wind, 1-3MW turbine, offshore']

In [71]:
[act for act in eidb if act["location"] == "CENTREL"]

['electricity, high voltage, production mix' (kilowatt hour, CENTREL, None)]

In [110]:
lca.inventory.shape[1]

16022

In [121]:
geomatcher.contained?

In [120]:
[act for act in eidb if act["location"] == "RER"]

['oxidation of butane' (kilogram, RER, None),
 'hydropower plant construction, reservoir, alpine region' (unit, RER, None),
 'brass drilling, conventional' (kilogram, RER, None),
 'cast iron milling, large parts' (kilogram, RER, None),
 'blast furnace production' (unit, RER, None),
 'market for cyanoacetic acid' (kilogram, RER, None),
 'acetaldehyde oxidation' (kilogram, RER, None),
 'laser machining, metal, with YAG-laser, 500W power' (hour, RER, None),
 'market for dipropylene glycol monomethyl ether' (kilogram, RER, None),
 'polystyrene production, general purpose' (kilogram, RER, None),
 'sawing and planing, azobe, air dried' (cubic meter, RER, None),
 'kaolin production' (kilogram, RER, None),
 'photovoltaic panel production, multi-Si wafer' (square meter, RER, None),
 'transport, freight, lorry 3.5-7.5 metric ton, EURO3' (ton kilometer, RER, None),
 'scrap preparation facility construction' (unit, RER, None),
 'impact extrusion of aluminium, deformation stroke' (kilogram, RER, No

In [135]:
eidb.name

'ecoinvent_3.5'

In [143]:
def bioflows_for_rmnd_tech(key, regions, db):
    techs = available_electricity_generating_technologies[key]
    if len(techs) == 0:
        return None
    print("Processing {} ({}) in {}".format(key, region, db.name))
    actvts = [act for act in db if act["name"] in techs and
                act["location"] in regions]
    if len(actvts) == 0:
        actvts = [act for act in carmadb if act["name"] in techs and
                    act["location"] == "RoW"]
        if len(actvts) == 0:
            actvts = [act for act in carmadb if act["name"] in techs and
                        act["location"] == "GLO"]
            if len(actvts) == 0:
                print("Could not find any activities matching {}".format(techs))
                return None
    
    lca = LCA({act: 1 for act in actvts})
    lca.lci()
    
    return lca
            

In [147]:
def rmnd_bioflows(region):
    carma_techs = ['Biomass IGCC CCS', 'Biomass IGCC', 'Coal IGCC', 'Coal IGCC CCS', 'Coal PC CCS', 'Gas CCS']
    carmadb = Database("Carma CCS")
    eidb = Database("ecoinvent_3.5")
    result = {}
    bio_names = []
    regions = [el[1] if type(el) == tuple else el for el in geomatcher.contained(("REMIND", region))]
    if region == "EUR":
        regions.append("RER")
    for key in available_electricity_generating_technologies:
        if len(available_electricity_generating_technologies[key]) == 0:
            print("No technologies available for {}! Skipping.".format(key))
            continue
        if key in carma_techs:
            lca = bioflows_for_rmnd_tech(key, regions, carmadb)
        else:
            lca = bioflows_for_rmnd_tech(key, regions, eidb)

        # bionames are only generated once. This takes some time.
        # probably these never change...
        if len(bio_names) != lca.inventory.shape[0]:
            print("Bionames changed, old length: {}".format(len(bio_names)))
            bio_names = [get_activity(key)["name"] for key in lca.biosphere_dict]
            print("new length: {}".format(len(bio_names)))

        result[key] = pd.DataFrame.from_dict({
            "flow": bio_names,
            "amount": [lca.inventory[n, :].sum() for n in range(len(bio_names))]
        }).groupby("flow").agg({"amount": sum})

    return pd.concat(result)

In [148]:
df = rmnd_bioflows("EUR")

Processing Biomass IGCC CCS (EUR) in Carma CCS
Bionames changed, old length: 0
new length: 2079
Processing Biomass IGCC (EUR) in Carma CCS
Processing Coal IGCC (EUR) in Carma CCS
Processing Coal IGCC CCS (EUR) in Carma CCS
Processing Coal PC CCS (EUR) in Carma CCS
Processing Gas CCS (EUR) in Carma CCS
Processing Biomass CHP (EUR) in ecoinvent_3.5
Bionames changed, old length: 2079
new length: 2077
Processing Coal PC (EUR) in ecoinvent_3.5
Processing Coal CHP (EUR) in ecoinvent_3.5
Processing Gas OC (EUR) in ecoinvent_3.5
Processing Gas CC (EUR) in ecoinvent_3.5
Processing Gas CHP (EUR) in ecoinvent_3.5
Processing Geothermal (EUR) in ecoinvent_3.5
Processing Hydro (EUR) in ecoinvent_3.5
No technologies available for Hydrogen! Skipping.
Processing Nuclear (EUR) in ecoinvent_3.5
Processing Oil (EUR) in ecoinvent_3.5
Processing Solar CSP (EUR) in ecoinvent_3.5
Processing Solar PV (EUR) in ecoinvent_3.5
Processing Wind (EUR) in ecoinvent_3.5


In [152]:
df.sortlevel().sample(20)

/home/al/miniconda3/envs/rmnd-lca/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  """Entry point for launching an IPython kernel.


,,amount
,flow,
Biomass IGCC CCS,Uranium-235,2.156287e-05
Solar CSP,"Cobalt, Co 5.0E-2%, in mixed ore, in ground",1.984838e-08
Gas CHP,"Transformation, to annual crop, irrigated, extensive",0.000000e+00
Wind,Cymoxanil,0.000000e+00
Coal IGCC CCS,"Kaolinite, 24% in crude ore, in ground",3.826368e-06
Coal PC,Radon-222,8.400717e+01
Biomass CHP,Organic carbon,1.466272e-06
Coal CHP,Oxamyl,0.000000e+00
Coal PC,"strontium, in ground",1.999745e-08
